In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from unirep_reimplementation import aa_seq_to_int
import jax.numpy as np
from fundl.layers.rnn import mlstm1900

sequence length:  238


/home/kummear1/anaconda/envs/fundl-dev/lib/python3.7/site-packages/jax/lib/xla_bridge.py:114: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


embedding shape:  (239, 10)
output:  [[ 6.03701472e-01  4.72046107e-01 -5.25433302e-01 ...  5.82538962e-01
   4.91422802e-01  5.87047517e-01]
 [ 3.03941277e-07 -5.36410153e-05 -4.01969942e-08 ... -6.70601265e-04
   1.27420835e-02 -2.79029626e-02]
 [ 7.54976791e-05 -8.29329249e-03 -9.17237834e-04 ...  1.57487695e-04
   4.46955906e-03  6.78823702e-03]
 ...
 [ 6.71174144e-04  2.76400615e-02  1.47543967e-01 ...  1.25471931e-02
   3.69270369e-02 -2.33784989e-02]
 [ 4.77424404e-03  2.77729742e-02  6.06863350e-02 ...  2.04841178e-02
   4.66152318e-02 -3.19362395e-02]
 [ 1.18345907e-03  3.06937192e-02  8.87814388e-02 ...  8.07420537e-03
   7.05695972e-02 -1.01826396e-02]]
reps:  [ 0.00497921 -0.04242463  0.04092563 ...  0.03462332  0.07152246
 -0.00351865]
output shape:  (239, 1900)
Time taken: 4.50 seconds


In [3]:
sequence = "MRKGEELFTGVVPILVELDGDVNGHKFSVRGEGEGDATNGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTISFKDDGTYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNFNSHNVYITADKQKNGIKANFKIRHNVEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSVLSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [4]:
from fundl.layers import dense
from fundl.activations import softmax

In [5]:
# load unirep pre-trained weights
params = dict()
params["unirep"] = dict()
params["unirep"]["gh"] = np.load("1900_weights/rnn_mlstm_mlstm_gh:0.npy")
params["unirep"]["gmh"] = np.load("1900_weights/rnn_mlstm_mlstm_gmh:0.npy")
params["unirep"]["gmx"] = np.load("1900_weights/rnn_mlstm_mlstm_gmx:0.npy")
params["unirep"]["gx"] = np.load("1900_weights/rnn_mlstm_mlstm_gx:0.npy")

params["unirep"]["wh"] = np.load("1900_weights/rnn_mlstm_mlstm_wh:0.npy")
params["unirep"]["wmh"] = np.load("1900_weights/rnn_mlstm_mlstm_wmh:0.npy")
params["unirep"]["wmx"] = np.load("1900_weights/rnn_mlstm_mlstm_wmx:0.npy")
params["unirep"]["wx"] = np.load("1900_weights/rnn_mlstm_mlstm_wx:0.npy")

params["unirep"]["b"] = np.load("1900_weights/rnn_mlstm_mlstm_b:0.npy")

In [6]:
from fundl.weights import add_dense_params

In [7]:
params = add_dense_params(params, "dense", input_dim=1900, output_dim=26)

In [8]:
params.keys()

dict_keys(['unirep', 'dense'])

In [9]:
from fundl.layers.rnn import mlstm1900

In [10]:
def model(p, x):
    x = mlstm1900(p['unirep'], x)
    x = dense(p['dense'], x, nonlin=softmax)
    return x

In [11]:
sequence = aa_seq_to_int(sequence)[:-1]

embeddings = np.load("1900_weights/embed_matrix:0.npy")
x = np.vstack([embeddings[i] for i in sequence])
print("embedding shape: ", x.shape)

embedding shape:  (239, 10)


In [12]:
model(params, x)

DeviceArray([[0.03974526, 0.04554541, 0.04773873, ..., 0.03906574,
              0.04798882, 0.02813344],
             [0.03951884, 0.03820375, 0.03702553, ..., 0.03749093,
              0.0407745 , 0.03895325],
             [0.03885591, 0.03452536, 0.0363565 , ..., 0.03999977,
              0.04054311, 0.04435818],
             ...,
             [0.03493968, 0.0370485 , 0.04554243, ..., 0.03557619,
              0.03681337, 0.045091  ],
             [0.03704586, 0.0379707 , 0.04004375, ..., 0.03477773,
              0.03667038, 0.03804887],
             [0.03817863, 0.03915823, 0.0396863 , ..., 0.03621582,
              0.0377139 , 0.04132376]], dtype=float32)

In [13]:
oh_mat = np.eye(26)
oh = {i: oh_mat[i] for i in range(26)}
y = np.vstack([[oh[i] for i in sequence[1:]], np.zeros(26)])

In [14]:
from jax import grad
from jax.experimental.optimizers import adam

In [15]:
init, update, get_params = adam(step_size=0.005)

In [16]:
from fundl.losses import neg_cross_entropy_loss

In [17]:
dloss = grad(neg_cross_entropy_loss)

In [18]:
import time

In [ ]:
state = init(params)
for i in range(1000):
    start= time.time()
    g = dloss(params, model, x, y)
    l = neg_cross_entropy_loss(params, model, x, y)

    state = update(i, g, state)
    params = get_params(state)
    
    print(i, l, 'time:', time.time() - start)

0 0.16258058 time: 37.79021239280701
1 0.15013629 time: 33.051103830337524
2 0.160756 time: 158.2134461402893
3 0.15814093 time: 238.71153569221497
4 0.14887147 time: 238.2558035850525
5 0.14526771 time: 329.3936948776245
6 0.14284708 time: 303.70419669151306
7 0.14056934 time: 297.2267339229584
8 0.13960335 time: 292.0242998600006
